In [1]:
%pip install -q paddlepaddle paddleseg EISeg

Note: you may need to restart the kernel to use updated packages.


In [2]:
!curl -LO https://github.com/saic-vul/fbrs_interactive_segmentation/releases/download/v1.0/Berkeley.zip \
    && unzip Berkeley.zip \
    && rm Berkeley.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:--  0:00:12 --:--:--     0     0      0 --:--:--  0:00:06 --:--:--     0
100 7072k  100 7072k    0     0   425k      0  0:00:16  0:00:16 --:--:-- 2310k4 2054k
Archive:  Berkeley.zip
   creating: Berkeley/
   creating: Berkeley/images/
  inflating: Berkeley/images/101085.jpg  
  inflating: Berkeley/images/101087.jpg  
  inflating: Berkeley/images/102061.jpg  
  inflating: Berkeley/images/106024.jpg  
  inflating: Berkeley/images/108005.jpg  
  inflating: Berkeley/images/108070.jpg  
  inflating: Berkeley/images/108082.jpg  
  inflating: Berkeley/images/109053.jpg  
  inflating: Berkeley/images/118035.jpg  
  inflating: Berkeley/images/12003.jpg  
  inflating: Berkeley/images/123074.jpg  
  inflating: Berkeley/images/124084.jpg  
  inflating: Berkeley/images/126007.jpg  
  inflatin

In [3]:
!curl -LO https://paddleseg.bj.bcebos.com/eiseg/0.4/static_hrnet18s_ocr48_cocolvis.zip \
    && unzip static_hrnet18s_ocr48_cocolvis.zip \
    && rm static_hrnet18s_ocr48_cocolvis.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 15.1M  100 15.1M    0     0  1356k      0  0:00:11  0:00:11 --:--:-- 2816k00:09  0:00:11 1195k
Archive:  static_hrnet18s_ocr48_cocolvis.zip
   creating: static_hrnet18s_ocr48_cocolvis/
  inflating: __MACOSX/._static_hrnet18s_ocr48_cocolvis  
  inflating: static_hrnet18s_ocr48_cocolvis/static_hrnet18s_ocr48_cocolvis.pdmodel  
  inflating: __MACOSX/static_hrnet18s_ocr48_cocolvis/._static_hrnet18s_ocr48_cocolvis.pdmodel  
  inflating: static_hrnet18s_ocr48_cocolvis/static_hrnet18s_ocr48_cocolvis.pdiparams  
  inflating: __MACOSX/static_hrnet18s_ocr48_cocolvis/._static_hrnet18s_ocr48_cocolvis.pdiparams  


In [4]:
from pathlib import Path

import cv2
import numpy as np
from paddle.io import Dataset


class BerkeleyDataset(Dataset):
    def __init__(self, dataset_path):
        self.dataset_path = Path(dataset_path)
        self._images_path = self.dataset_path / "images"
        self._masks_path = self.dataset_path / "masks"

        self.dataset_samples = [x.name for x in sorted(self._images_path.glob('*.*'))]
        self._masks_paths = {x.stem: x for x in self._masks_path.glob('*.*')}
        
    def __getitem__(self, index):
        image_name = self.dataset_samples[index]
        image_path = str(self._images_path / image_name)
        mask_path = str(self._masks_paths[image_name.split('.')[0]])

        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        instances_mask = cv2.imread(mask_path)[:, :, 0].astype(np.int32)
        instances_mask[instances_mask == 128] = -1
        instances_mask[instances_mask > 128] = 1
        return image, instances_mask
    
    def __len__(self):
        return len(self.dataset_samples)

In [5]:
from eiseg.inference import clicker


class PredictorWrapper:
    def __init__(self, predictor):
        self.predictor = predictor
        self.clicker = clicker.Clicker()
        
    def predict(self, click):
        self.clicker.add_click(clicker.Click(is_positive=click.is_positive, coords=(click.y, click.x)))
        prediction = self.predictor.get_prediction(self.clicker)
        return prediction > 0.49
        
        
class PaddleModelFactory:
    def __init__(self, predictor):
        self.predictor = predictor
        
    def get_model(self, image):
        self.predictor.set_input_image(image)
        return PredictorWrapper(self.predictor)

In [6]:
import paddle.inference as paddle_infer
from eiseg.inference.predictor import get_predictor

from isegeval import evaluate


dataset = BerkeleyDataset("./Berkeley")

base_path = "./static_hrnet18s_ocr48_cocolvis"
model_path = base_path + "/static_hrnet18s_ocr48_cocolvis.pdmodel"
param_path = base_path + "/static_hrnet18s_ocr48_cocolvis.pdiparams"

config = paddle_infer.Config(model_path, param_path)
config.enable_mkldnn()
config.switch_ir_optim(True)
config.set_cpu_math_library_num_threads(10)

model = paddle_infer.create_predictor(config)
predictor_params = {
    "brs_mode": "NoBRS",
    "with_flip": False,
    "zoom_in_params": {
        "skip_clicks": -1,
        "target_size": (400, 400),
        "expansion_ratio": 1.4,
    },
    "predictor_params": {
        "net_clicks_limit": None,
        "max_size": 800,
        "with_mask": True,
    },
}

predictor = get_predictor(model, **predictor_params)
model_factory = PaddleModelFactory(predictor)

print("NoC@90: ", evaluate(dataset, model_factory))

E1214 14:48:41.435971 259745280 analysis_config.cc:563] Please compile with MKLDNN first to use MKLDNN
--- Running analysis [ir_graph_build_pass]
--- Running analysis [ir_graph_clean_pass]
--- Running analysis [ir_analysis_pass]
--- Running IR pass [simplify_with_basic_ops_pass]
--- Running IR pass [layer_norm_fuse_pass]
---    Fused 0 subgraphs into layer_norm op.
--- Running IR pass [attention_lstm_fuse_pass]
--- Running IR pass [seqconv_eltadd_relu_fuse_pass]
--- Running IR pass [seqpool_cvm_concat_fuse_pass]
--- Running IR pass [mul_lstm_fuse_pass]
--- Running IR pass [fc_gru_fuse_pass]
---    fused 0 pairs of fc gru patterns
--- Running IR pass [mul_gru_fuse_pass]
--- Running IR pass [seq_concat_fc_fuse_pass]
--- Running IR pass [gpu_cpu_squeeze2_matmul_fuse_pass]
--- Running IR pass [gpu_cpu_reshape2_matmul_fuse_pass]
--- Running IR pass [gpu_cpu_flatten2_matmul_fuse_pass]
--- Running IR pass [matmul_v2_scale_fuse_pass]
--- Running IR pass [gpu_cpu_map_matmul_v2_to_mul_pass]
--- 

NoC@90:  3.27
